In [3]:
import pygame
import random
import os

# Inicializa o Pygame
pygame.init()

# Definir as cores
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Tamanho da tela
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# Definindo o tamanho da tela
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Space Shooter")

# Função para carregar o recorde
def load_highscore():
    if os.path.exists("highscore.txt"):
        with open("highscore.txt", "r") as f:
            return int(f.read())
    return 0

# Função para salvar o recorde
def save_highscore(score):
    with open("highscore.txt", "w") as f:
        f.write(str(score))

# Classe para a nave
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface([50, 50])
        self.image.fill(BLUE)
        self.rect = self.image.get_rect()
        self.rect.center = (SCREEN_WIDTH // 2, SCREEN_HEIGHT - 50)
        self.speed = 5

    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and self.rect.x > 0:
            self.rect.x -= self.speed
        if keys[pygame.K_RIGHT] and self.rect.x < SCREEN_WIDTH - self.rect.width:
            self.rect.x += self.speed

# Classe para os inimigos
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface([40, 40])
        self.image.fill(RED)
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(0, SCREEN_WIDTH - self.rect.width)
        self.rect.y = random.randint(-100, -40)
        self.speed = random.randint(3, 6)

    def update(self):
        self.rect.y += self.speed
        if self.rect.y > SCREEN_HEIGHT:
            self.rect.y = random.randint(-100, -40)
            self.rect.x = random.randint(0, SCREEN_WIDTH - self.rect.width)

# Classe para os tiros
class Bullet(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface([5, 10])
        self.image.fill(WHITE)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.speed = 7

    def update(self):
        self.rect.y -= self.speed
        if self.rect.y < 0:
            self.kill()

# Função para desenhar o menu de pausa
def draw_pause_menu():
    font = pygame.font.SysFont("Arial", 40)
    text_continue = font.render("Pressione C para continuar", True, WHITE)
    text_reset = font.render("Pressione R para resetar", True, WHITE)
    text_exit = font.render("Pressione Q para sair", True, WHITE)
    screen.fill(BLACK)
    screen.blit(text_continue, (SCREEN_WIDTH // 2 - text_continue.get_width() // 2, SCREEN_HEIGHT // 2 - 50))
    screen.blit(text_reset, (SCREEN_WIDTH // 2 - text_reset.get_width() // 2, SCREEN_HEIGHT // 2))
    screen.blit(text_exit, (SCREEN_WIDTH // 2 - text_exit.get_width() // 2, SCREEN_HEIGHT // 2 + 50))
    pygame.display.flip()

# Função principal
def game():
    # Definir o clock para controlar a taxa de atualização
    clock = pygame.time.Clock()

    # Grupo de sprites
    all_sprites = pygame.sprite.Group()
    bullets = pygame.sprite.Group()
    enemies = pygame.sprite.Group()

    # Criar o jogador
    player = Player()
    all_sprites.add(player)

    # Criar inimigos (inimigos infinitos)
    for i in range(10):
        enemy = Enemy()
        all_sprites.add(enemy)
        enemies.add(enemy)

    # Pontuação
    score = 0
    highscore = load_highscore()
    font = pygame.font.SysFont("Arial", 30)

    # Loop principal do jogo
    running = True
    paused = False  # Variável de controle para pausa

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and not paused:
                    # Disparar uma bala
                    bullet = Bullet(player.rect.centerx, player.rect.top)
                    all_sprites.add(bullet)
                    bullets.add(bullet)
                elif event.key == pygame.K_ESCAPE:
                    # Ativa a pausa
                    paused = True
                elif event.key == pygame.K_c and paused:
                    # Continuar o jogo
                    paused = False
                elif event.key == pygame.K_r and paused:
                    # Resetar o jogo
                    return game()  # Reinicia o jogo
                elif event.key == pygame.K_q and paused:
                    # Sair do jogo
                    running = False

        if not paused:
            # Atualiza todos os sprites
            all_sprites.update()

            # Verificar colisões entre tiros e inimigos
            for bullet in bullets:
                enemy_hits = pygame.sprite.spritecollide(bullet, enemies, True)
                for enemy in enemy_hits:
                    score += 10
                    bullet.kill()
                    # Recria um novo inimigo
                    enemy = Enemy()
                    all_sprites.add(enemy)
                    enemies.add(enemy)

            # Verificar colisões entre o jogador e os inimigos
            if pygame.sprite.spritecollideany(player, enemies):
                running = False
                # Verifica se a pontuação é maior que o recorde
                if score > highscore:
                    highscore = score
                    save_highscore(highscore)

            # Preenche a tela com a cor de fundo
            screen.fill(BLACK)

            # Desenha todos os sprites
            all_sprites.draw(screen)

            # Exibe a pontuação e o recorde
            score_text = font.render(f"Score: {score}", True, WHITE)
            highscore_text = font.render(f"Highscore: {highscore}", True, GREEN)
            screen.blit(score_text, [10, 10])
            screen.blit(highscore_text, [SCREEN_WIDTH - highscore_text.get_width() - 10, 10])

        # Se o jogo estiver pausado, desenha o menu de pausa
        if paused:
            draw_pause_menu()

        # Atualiza a tela
        pygame.display.flip()

        # Controla os FPS
        clock.tick(60)

    pygame.quit()

# Iniciar o jogo
if __name__ == "__main__":
    game()
